we wanna aggregate all datasets in one clean data set 
### columns are : 
company         : 1-ONCF,2-CTM,3-TAXI,4-Covoiturage

from_lat        : latitude of the origine city

from_lan        : langitude of the origine city

to_lat          : latitude of the destination city

to_lan          : langitude of the destination city

distance        : distance between origin and destination

duration        : traveling duration

price           : travling cost


In [1]:
import numpy as np
import requests
import json
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import re
import datetime
from datetime import timedelta
from sklearn.linear_model import LinearRegression

In [2]:
def getLocationOfCity(city):
    api_key = '966a766bb88e4cb5a5f78ffd9ec99208'
    req = requests.get('https://api.opencagedata.com/geocode/v1/json?q='+city+'&key='+api_key)
    js = json.loads(req.text)
    rep = None
    for res in js['results']:
        if res['formatted'].split(',')[-1] == " Morocco":
            rep = res['geometry'] 
            break
    return rep

In [3]:
def distance(locationA,locationB):
    R = 6373.0

    lat1 = radians(locationA['lat'])
    lon1 = radians(locationA['lng'])
    lat2 = radians(locationB['lat'])
    lon2 = radians(locationB['lng'])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [4]:
oncfDF = pd.read_csv("oncfTrips.csv",delimiter=";",encoding='utf-8')
ctmDF = pd.read_csv("CTMTrips.csv",delimiter=";",encoding='latin-1')
covDF = pd.read_csv("cov.csv",delimiter=",",encoding='latin-1')

In [5]:
cities = list(oncfDF['from'])
cities.extend(list(oncfDF['to']))
cities.extend(list(ctmDF['from']))
cities.extend(list(ctmDF['to']))

In [6]:
cities = list(set(cities))
cities = [city.upper() for city in cities]
cities = list(set(cities))

In [7]:
locations = {}
for city in cities:
    locations[city] = getLocationOfCity(city)

In [8]:
locations['SIDI SLIMANE MEDINA']={'lat': 34.2737827,'lng':-5.9806831}
locations['KELAA  DES  SRAGHNAS']={'lat' :32.0510291,'lng':-7.4206207}
locations['GUELMIMA']={'lat' :31.6844986,'lng':-4.979725}
locations['AEROPORT MED V']={'lat' :33.3699749,'lng':-7.5879118}

In [9]:
def duration(time1,time2):
    FMT = '%H:%M'
    H1 = time1.split(':')[0]
    H2 = time2.split(':')[0]
    date1 = datetime.datetime.strptime(time1,FMT)
    date2 = datetime.datetime.strptime(time2,FMT) 
    if int(H2) < int(H1):
        date2 += timedelta(days=1)
    diff = date2 - date1
    return diff.seconds / 60

In [10]:
tfrom = oncfDF['from']
tto = oncfDF['to']
tdepart = oncfDF['depart_time']
tarrival = oncfDF['arrival_time']
tprice = oncfDF['price']
data = []
for i in range(len(tfrom)):
    data.append([1,locations[tfrom[i].upper()]['lat'],locations[tfrom[i].upper()]['lng'],locations[tto[i].upper()]['lat'],locations[tto[i].upper()]['lng'],
               distance(locations[tfrom[i].upper()],locations[tto[i].upper()]),
               tdepart[i],duration(tdepart[i],tarrival[i]),int(tprice[i][:-3])])
    
    

In [11]:
tfrom = ctmDF['from']
tto = ctmDF['to']
tdepart = ctmDF['depart_time']
tarrival = ctmDF['arrival_time']
tprice = ctmDF['price']
tprice.fillna((tprice.mean()), inplace=True)
for i in range(len(tfrom)):
    data.append([2,locations[tfrom[i].upper()]['lat'],locations[tfrom[i].upper()]['lng'],locations[tto[i].upper()]['lat'],locations[tto[i].upper()]['lng'],
               distance(locations[tfrom[i].upper()],locations[tto[i].upper()]),
                tdepart[i],duration(tdepart[i],tarrival[i]),int(tprice[i])])

In [16]:
def timeFromDate(x):
    return x.split(' ')[1]

def minute(x):
    return  60*int(x.split(':')[0]) + int(x.split(':')[1])

In [18]:
# from_lat,from_lon,to_lat,to_lon,distance,duree,date_depart,prix
tfrom_lat = covDF['from_lat']
tfrom_lon = covDF['from_lon']
tto_lat = covDF['to_lat']
tto_lon = covDF['to_lon']
tprice = covDF['prix']
tdistance = covDF['distance']
tduree = covDF['duree']
tdepart = covDF['date_depart']
for i in range(len(tfrom_lat)):
    data.append([3,tfrom_lat[i],tfrom_lon[i],tto_lat[i],tto_lon[i],tdistance[i],timeFromDate(tdepart[i]),minute(tduree[i]),tprice[i]])

In [19]:
X = np.array([86.9,52.8,59.7,64.2,41.3,78.2,64.7,141]).reshape(-1,1)
y = np.array([63,44,68,48,39,57,54,97])
duration_model = LinearRegression().fit(X, y)
duration_model.score(X, y)

0.8443326971108013

In [20]:
def durationFromLoc(locationA,locationB):
    d = distance(locationA,locationB)
    return duration_model.predict([[d]])[0]  

In [21]:
durationFromLoc(locations['MEKNES'],locations['FES'])

45.86008621137471

In [24]:
len(data)

654801

In [25]:
X = np.array([202,54,91,119,60.2,67.5,63.9,24.2,78.1,188,28.8,115,44,126,124,154,85.4,25.3,51,112,128,93.8]).reshape(-1,1)
y = np.array([75,25,35,45,28,25,25,8,35,80,12,50,15,60,55,70,35,10,40,60,50,70])
price_model = LinearRegression().fit(X, y)
price_model.score(X, y)

0.8412497123562451

In [26]:
import random
for a in locations.values():
    for b in locations.values():
        if a!=b:
            data.append([4,a['lat'],a['lng'],b['lat'],b['lng'],distance(a,b),timeFromDate(random.choice(tdepart)),durationFromLoc(a,b),price_model.predict([[distance(a,b)]])[0]])
    

In [27]:
len(data)

661603

In [40]:
pd.DataFrame(data).to_csv('dataset.csv',sep=';',header=["company" ,"from_lat" ,"from_lan" ,"to_lat" ,"to_lan" ,"distance" ,"depart","duration" ,"price"])

In [29]:
def f(x):
    x = int(x.split(":")[0])
    if (x > 4) and (x <= 8):
        return 1
    elif (x > 8) and (x <= 12 ):
        return 2
    elif (x > 12) and (x <= 16):
        return 3
    elif (x > 16) and (x <= 20) :
        return 4
    elif (x > 20) and (x <= 24):
        return 5
    elif (x <= 4):
        return 0
def g(x):
    if x == 'M':
        return 1
    return 2

def h(x):
    p = ['technician', 'other', 'writer', 'executive', 'administrator',
       'student', 'lawyer', 'educator', 'scientist', 'entertainment',
       'programmer', 'librarian', 'homemaker', 'artist', 'engineer',
       'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor']
    for a,b in enumerate(p):
        if x  == b:
            return a
    

In [41]:
trips = pd.read_csv('../datasources/trips.csv',delimiter=";",encoding='utf-8')
users = pd.read_csv('../datasources/users.csv',delimiter=";",encoding='utf-8')
users.head()

,id,age,sex,profition
0,1,24,M,technician
1,2,53,F,other
2,3,23,M,writer
3,4,24,M,technician
4,5,33,F,other


In [42]:
trips["session"] = trips['depart'].apply(f)
trips = trips.drop(axis=1,columns=['depart'])
users['profition'] = users['profition'].apply(h)
users['sex'] = users['sex'].apply(g)

In [43]:
trips['id']  

0              0
1              1
2              2
3              3
4              4
5              5
6              6
7              7
8              8
9              9
10            10
11            11
12            12
13            13
14            14
15            15
16            16
17            17
18            18
19            19
20            20
21            21
22            22
23            23
24            24
25            25
26            26
27            27
28            28
29            29
           ...  
661573    661573
661574    661574
661575    661575
661576    661576
661577    661577
661578    661578
661579    661579
661580    661580
661581    661581
661582    661582
661583    661583
661584    661584
661585    661585
661586    661586
661587    661587
661588    661588
661589    661589
661590    661590
661591    661591
661592    661592
661593    661593
661594    661594
661595    661595
661596    661596
661597    661597
661598    661598
661599    661599
661600    6616

In [44]:
n= np.random.randint(500,701)
dd = trips.sample(n,random_state=1)
for j in range(1,5):
    tmp = dd.loc[dd['company']==j]
    p = len(tmp)
    for k,l in zip(range(p),tmp['id']):
        print(k,l)

0 4218
1 2484
2 2762
3 13050
4 4917
5 1149
6 2636
7 6423
8 3335
9 5430
10 6070
11 5451
12 4034
0 14166
1 15378
2 16795
3 16051
0 371683
1 210123
2 278799
3 344628
4 141821
5 212778
6 76534
7 569443
8 520961
9 395253
10 140514
11 465653
12 547118
13 218421
14 364883
15 583203
16 297712
17 213859
18 382923
19 398085
20 282776
21 310648
22 284141
23 487125
24 591728
25 171942
26 624001
27 649155
28 106587
29 196197
30 106434
31 114621
32 501310
33 412215
34 308463
35 422406
36 601386
37 374680
38 191155
39 640988
40 525500
41 628287
42 417187
43 99045
44 627584
45 393778
46 389818
47 115132
48 610530
49 318747
50 305328
51 429754
52 318253
53 134206
54 155735
55 365574
56 446619
57 137452
58 231421
59 366949
60 623222
61 564411
62 530890
63 232481
64 294341
65 294138
66 392630
67 580606
68 159056
69 498741
70 245911
71 146616
72 250667
73 443487
74 580489
75 266601
76 647208
77 105729
78 445581
79 588013
80 649237
81 356786
82 418072
83 640286
84 417880
85 543056
86 350495
87 202680
88 46

In [49]:
hist = []
m = len(users)
for i in range(m):
    n = np.random.randint(500,701)
    dd = trips.sample(n,random_state=1)
    for j in range(1,5):
        tmp = dd.loc[dd['company']==j]
        p = len(tmp)
        if j == 1:
            security = np.full(shape=p,fill_value=1)
            trafic = np.full(shape=p,fill_value=0)
            retard = np.round(np.random.normal(0.3, 0.1, p) * 60)
        else :
            security = np.round(np.random.normal(0.55, 0.1, p) * 1)
            trafic = np.round(np.random.normal(0.4, 0.1, p) * 1)
            retard = np.round(np.random.normal(0.3, 0.1, p) * 120)
        comfort = np.round(np.random.normal(0.55, 0.1, p) * 1)
        speed = np.round(np.random.normal(0.5, 0.2, p) * 2)
        for k,l in zip(range(p),tmp['id']):
            hist.append([users['id'][i],tmp['id'][l],comfort[k],speed[k],trafic[k],security[k],retard[k]])
            
            
       

In [50]:
pd.DataFrame(hist).to_csv('history.csv',sep=';',header=["user_id","trip_id","comfort","speed","trafic","security","retard"],index= False)

In [51]:
hist = pd.read_csv('history.csv',delimiter=";",encoding='utf-8')

In [52]:
len(hist)

568878

In [76]:
trips = pd.read_csv('../datasources/trips.csv',delimiter=';')
users = pd.read_csv('../datasources/users.csv',delimiter=';')
history = pd.read_csv('../datasources/history.csv',delimiter=';')
trips.head()

,id,company,from_lat,from_lan,to_lat,to_lan,distance,depart,duration,price
0,0,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,07:50,47.0,66.0
1,1,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,07:50,90.0,50.0
2,2,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,11:50,47.0,66.0
3,3,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,11:50,90.0,50.0
4,4,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,16:50,47.0,66.0


In [56]:
def f(x):
    x = int(x.split(":")[0])
    if (x > 4) and (x <= 8):
        return 1
    elif (x > 8) and (x <= 12 ):
        return 2
    elif (x > 12) and (x <= 16):
        return 3
    elif (x > 16) and (x <= 20) :
        return 4
    elif (x > 20) and (x <= 24):
        return 5
    elif (x <= 4):
        return 0
def g(x):
    if x == 'M':
        return 1
    return 2

def h(x):
    p = ['technician', 'other', 'writer', 'executive', 'administrator',
       'student', 'lawyer', 'educator', 'scientist', 'entertainment',
       'programmer', 'librarian', 'homemaker', 'artist', 'engineer',
       'marketing', 'none', 'healthcare', 'retired', 'salesman', 'doctor']
    for a,b in enumerate(p):
        if x  == b:
            return a

def ff(x):
    if x<30 :
        return 0
    elif x<60:
        return 1
    elif x<90:
        return 2
    else:
        return 3

In [77]:
trips["session"] = trips['depart'].apply(f)


In [78]:
trips = trips.drop(axis=1,columns=['depart','id'])


In [79]:
users['profition'] = users['profition'].apply(h)


In [80]:
users['sex'] = users['sex'].apply(g)


In [81]:
users = users.drop(axis=1,columns=['id'])


In [83]:
trips.head()

,company,from_lat,from_lan,to_lat,to_lan,distance,duration,price,session
0,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,47.0,66.0,1
1,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,90.0,50.0,1
2,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,47.0,66.0,2
3,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,90.0,50.0,2
4,1,33.369975,-7.587912,34.229627,-6.528318,136.873942,47.0,66.0,3


In [62]:
rating = {}

rating["00000"] = 1
rating["00001"] = 0
rating["00002"] = 0
rating["00003"] = 0

rating["00010"] = 2
rating["00011"] = 1
rating["00012"] = 0
rating["00013"] = 0

rating["00100"] = 1
rating["00101"] = 0
rating["00102"] = 0
rating["00103"] = 0

rating["00110"] = 1
rating["00111"] = 1
rating["00112"] = 0
rating["00113"] = 0

rating["01000"] = 2
rating["01001"] = 1
rating["01002"] = 0
rating["01003"] = 0

rating["01010"] = 3
rating["01011"] = 2
rating["01012"] = 1
rating["01013"] = 0

rating["01100"] = 2
rating["01101"] = 1
rating["01102"] = 0
rating["01103"] = 0

rating["01110"] = 3
rating["01111"] = 2
rating["01112"] = 1
rating["01113"] = 0

rating["02000"] = 1
rating["02001"] = 0
rating["02002"] = 0
rating["02003"] = 0

rating["02010"] = 3
rating["02011"] = 2
rating["02012"] = 1
rating["02013"] = 0

rating["02100"] = 1
rating["02101"] = 0
rating["02102"] = 0
rating["02103"] = 0

rating["02110"] = 3
rating["02111"] = 2
rating["02112"] = 1
rating["02113"] = 0

rating["10000"] = 3
rating["10001"] = 2
rating["10002"] = 1
rating["10003"] = 0

rating["10010"] = 4
rating["10011"] = 3
rating["10012"] = 2
rating["10013"] = 1

rating["10100"] = 3
rating["10101"] = 2
rating["10102"] = 1
rating["10103"] = 0

rating["10110"] = 4
rating["10111"] = 3
rating["10112"] = 2
rating["10113"] = 1

rating["11000"] = 4
rating["11001"] = 3
rating["11002"] = 2
rating["11003"] = 1

rating["11010"] = 5
rating["11011"] = 4
rating["11012"] = 3
rating["11013"] = 2

rating["11100"] = 4
rating["11101"] = 3
rating["11102"] = 2
rating["11103"] = 1

rating["11110"] = 5
rating["11111"] = 4
rating["11112"] = 3
rating["11113"] = 2

rating["12000"] = 4
rating["12001"] = 3
rating["12002"] = 2
rating["12003"] = 1

rating["12010"] = 5
rating["12011"] = 4
rating["12012"] = 3
rating["12013"] = 2

rating["12100"] = 4
rating["12101"] = 3
rating["12102"] = 2
rating["12103"] = 1

rating["12110"] = 5
rating["12111"] = 4
rating["12112"] = 3
rating["12113"] = 2

history['retard'] = history['retard'].apply(ff)

In [64]:
rt = []
for i in range(len(history)):
    try :
        rt.append([history['user_id'][i],history['trip_id'][i],rating[str(int(history['comfort'][i]))+str(int(history['speed'][i]))+str(int(history['trafic'][i]))+str(int(history['security'][i]))+str(history['retard'][i])]])
    except :
        pass

In [65]:
X = []
trips = trips.to_numpy()
users = users.to_numpy()
for r in rt:
    X.append(np.concatenate([trips[r[1]],users[r[0]-1]]))

In [84]:
data = pd.DataFrame(X,columns=['company', 'from_lat', 'from_lan', 'to_lat', 'to_lan', 'distance',
       'duration', 'price','session','age', 'sex', 'profition'])

In [86]:
data['rating'] = np.array(rt)[:,2]

In [87]:
data.head()

,company,from_lat,from_lan,to_lat,to_lan,distance,duration,price,session,age,sex,profition,rating
0,1.0,31.511828,-9.762090,31.625826,-7.989161,168.494895,195.0,80.0,3.0,24.0,1.0,0.0,3
1,1.0,33.781005,-7.161029,34.044889,-6.814017,43.446220,39.0,24.0,4.0,24.0,1.0,0.0,5
2,1.0,33.599383,-7.612313,33.998312,-6.854291,82.933743,60.0,40.0,3.0,24.0,1.0,0.0,5
3,1.0,35.771443,-5.786140,34.230155,-4.010104,235.737054,417.0,148.0,3.0,24.0,1.0,0.0,5
4,1.0,34.034653,-5.016193,34.044889,-6.814017,165.708180,161.0,85.0,2.0,24.0,1.0,0.0,5


In [88]:
data.to_csv("dataset.csv",sep=';',index = False)
